# プロダクトマニュアルをVector Search Indexで検索可能な状態にする(標準時間:30分)

## 本ノートブックの目的：Databricks における Vector Search のテキスト検索と Index へのガバナンスを理解する
Q1. プロダクトマニュアルの前処理<br>
Q2. Vector Search Index の作成<br>
Q3. Vector Search のツール化とアクセス制御

## 事前準備 (標準時間：10分)

In [0]:
%pip install -U bs4 langchain_core langchain_text_splitters databricks-vectorsearch pydantic transformers unitycatalog-ai[databricks] unitycatalog-langchain[databricks] databricks-langchain

In [0]:
%restart_python

In [0]:
%run ./00_config

In [0]:
# 本ノートブックで利用するスキーマを作成
schema_name = f"05_vector_search_index_for_{user_name}"
print(f"schema_name: `{schema_name}`")
spark.sql(
    f"""
    CREATE SCHEMA IF NOT EXISTS {catalog_name}.{schema_name}
    """
)
spark.sql(f"USE {catalog_name}.{schema_name}")

# 本ノートブックで利用するボリュームを作成
volume_name = f"product_manual"
print(f"volume_name: `{volume_name}`")
spark.sql(
    f"""
    CREATE VOLUME IF NOT EXISTS {catalog_name}.{schema_name}.{volume_name}
    """
)

### ソースデータファイルのダウンロードとボリュームへのアップロード
1. 現在のノートブックの左型タブにある`Workspace (Ctrl + Alt + E)`を選択し、現在のディレクトリにある`data`フォルダに移動
1. `product_manual`フォルダのハンバーガーメニュー（`︙`）を選択して、 `ダウンロード形式：` -> `ソースファイル` をクリックしてデータファイルをダウンロード
1. ダウンロードした Zip ファイルを解凍
1. 現在のノートブックの左型タブにある`Catalog (Ctrl + Alt + C)`を選択後、`05_vector_search_index_for_{username}`スキーマ下にある`product_manual` Volume にてハンバーガーメニュー（`︙`）を選択し、`このボリュームにアップロード`を選択
1. 表示されたウィンドウに解凍した CSV ファイルをすべて配置して、`アップロード`を選択
1. 下記のセルを実行し、ファイルが配置されていることを確認

In [0]:
# ファイルが配置されていることを確認
src_file_dir = f"/Volumes/{catalog_name}/{schema_name}/{volume_name}"
file_list = dbutils.fs.ls(src_file_dir)
if file_list == []:
    raise Exception("ファイルがディレクトリにありません。ファイルを配置してください。")
display(dbutils.fs.ls(src_file_dir))

## Q1. プロダクトマニュアルの前処理(標準時間：10分)
ダウンロード後に解凍した`product_manual`フォルダから、`index.html`を開いてみましょう。 \
マニュアルはHTMLで記述され、FAQやトラブルシューティングといったセクションに分かれています。 

起票されたサポートチケットに対してマニュアルに基づいて適切な回答を作成するためには、マニュアルの記載をLLMに入力して適切な回答を考えてもらう必要があります。 \
しかし、膨大なマニュアルの全てをLLMへ入力することはモデルでは通常入力可能なトークンの数が決まっているため不可能であり、コストや性能の観点からも望ましくありません。

このため、マニュアルを小さな塊に分割し、クエリと関連しそうな記述の塊（一般的にチャンクと呼ばれます）を抽出して、LLMへ与える文脈情報とします。今回はHTMLのH2タグによってHTMLファイルを分割します。

### 実践例(langchainの公式ドキュメントから引用)
HTMLの構造は以下のようにタグとその中のコンテンツを区別して簡潔に説明できます：

1. h1タグ: 最も重要な大見出し
   - ページの主題やタイトルを表す
   - 通常、1ページに1つのみ使用

2. h2タグ: 中見出し
   - 主要なセクションや章を表す
   - h1の下位階層として複数使用可能

3. h3タグ: 小見出し
   - h2の下位階層として使用
   - セクション内の重要なポイントを表す

4. pタグ: 段落
   - 本文のテキストを含む
   - 見出しの下に配置され、関連するコンテンツを表示

今回はBeautifulSoupと呼ばれるHTMLの解析を行うPythonパッケージを用いて、マニュアルの各ファイルを分割し、テーブル形式で保存します。

In [0]:
html_string = """
<!DOCTYPE html>
<html lang='ja'>
<head>
  <meta charset='UTF-8'>
  <meta name='viewport' content='width=device-width, initial-scale=1.0'>
  <title>おしゃれなHTMLページの例</title>
</head>
<body>
  <h1>メインタイトル</h1>
  <p>これは基本的な内容を含む導入段落です。</p>
  
  <h2>セクション1：はじめに</h2>
  <p>このセクションではトピックを紹介します。以下はリストです：</p>
  <ul>
    <li>最初の項目</li>
    <li>2番目の項目</li>
    <li><strong>太字のテキスト</strong>と<a href='#'>リンク</a>を含む3番目の項目</li>
  </ul>
  
  <h3>サブセクション1.1：詳細</h3>
  <p>このサブセクションでは追加の詳細を提供します。以下は表です：</p>
  <table border='1'>
    <thead>
      <tr>
        <th>ヘッダー1</th>
        <th>ヘッダー2</th>
        <th>ヘッダー3</th>
      </tr>
    </thead>
    <tbody>
      <tr>
        <td>1行目、1列目</td>
        <td>1行目、2列目</td>
        <td>1行目、3列目</td>
      </tr>
      <tr>
        <td>2行目、1列目</td>
        <td>2行目、2列目</td>
        <td>2行目、3列目</td>
      </tr>
    </tbody>
  </table>

  <h2>結論</h2>
  <p>これは文書の結論です。</p>
</body>
</html>


"""

In [0]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from bs4 import BeautifulSoup
from transformers import OpenAIGPTTokenizer

max_chunk_size = 500
tokenizer = OpenAIGPTTokenizer.from_pretrained("openai-gpt")
text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(tokenizer, chunk_size=max_chunk_size, chunk_overlap=50)

def split_html_with_h3_p(html, min_chunk_size=20, max_chunk_size=500):
    """
    HTMLコンテンツをh2, h3, pタグに基づいてチャンクに分割する関数。
    分割後、チャンクがmin_chunk_size以上のトークン数であるものだけを返す。

    Args:
        html (str): 分割対象のHTML文字列
        min_chunk_size (int): 分割後のチャンクが保持すべき最小トークン数
        max_chunk_size (int): 各チャンクの最大トークン数

    Returns:
        list[str]: 分割されたテキストチャンクのリスト
    """

    if not html:
        return []

    soup = BeautifulSoup(html, "html.parser")
    chunks = []
    current_h2 = None
    current_h3 = None
    current_chunk = ""

    for tag in soup.find_all(["h2", "h3", "p"]):
        text = tag.get_text(strip=True)
        if not text:
            continue

        if tag.name == "h2":
            # 新しいh2タグが現れた場合、既存のチャンクを分割してリストに追加。
            if current_chunk:
                chunks.extend(text_splitter.split_text(current_chunk.strip()))
            current_h2 = text
            current_h3 = None  # h3をリセット
            current_chunk = f"{current_h2}\n"
            
        elif tag.name == "p":
            # pは現在のh2またはh3に追加
            if len(tokenizer.encode(current_chunk + text)) <= max_chunk_size:
                current_chunk += text + "\n"
            else:
                chunks.extend(text_splitter.split_text(current_chunk.strip()))
                current_chunk = text + "\n"

    if current_chunk:
        chunks.extend(text_splitter.split_text(current_chunk.strip()))

    return [c for c in chunks if len(tokenizer.encode(c)) > min_chunk_size]

In [0]:
display(split_html_with_h3_p(html_string))

### ToDo: HTMLファイルのレコードをチャンクへ分割し、`product_documentation`テーブルへ格納

In [0]:
%sql
-- Change Data Feed を有効にする必要があるため、インデックス作成前に設定する

CREATE TABLE IF NOT EXISTS product_documentation (
  id BIGINT GENERATED BY DEFAULT AS IDENTITY,
  filename STRING,
  content STRING
) TBLPROPERTIES (delta.enableChangeDataFeed = true); 

In [0]:
import pyspark.sql.functions as F

html_file = (
    spark
    .read
    .option("wholetext", True)
    .text(f"/Volumes/{catalog_name}/{schema_name}/{volume_name}/*.html")
    .withColumn("filename", F.col("_metadata.file_path"))
    .withColumn("filename", F.regexp_replace(F.col("filename"), r'^.*/', ''))
    )

display(html_file)

In [0]:
import pandas as pd

@F.pandas_udf("array<string>")
def parse_and_split(docs: pd.Series) -> pd.Series:
    return docs.apply(split_html_with_h3_p)

In [0]:
(
    html_file
    .withColumn("content", F.explode(parse_and_split("value")))
    .select("filename", "content")
    .write.mode('overwrite').saveAsTable("product_documentation")
)

In [0]:
%sql
SELECT * FROM product_documentation

## Q2. Vector Search Index の作成(標準時間：10分)

### ベクトル検索とは
![ベクトル検索とは](/Workspace/Users/shotaro.kotani@nssoldlp.onmicrosoft.com/openhack2025/contents_with_answer/dev_shotkotani/data/img/vector_index.png)

この画像は、ベクトル検索の概念をわかりやすく表現しています。

*   **ベクトル表現（赤い点）:** 各赤い点は、特定のデータオブジェクト（例えば、画像、テキストドキュメント）のベクトル表現を表しています。ディープラーニングモデルなどの手法を用いて、データオブジェクトはその意味や内容に基づいて数値ベクトルに変換されます。
*   **クラスタリング:** 同様のデータオブジェクトは、多次元空間内で互いに近い位置に集まり、クラスタを形成します。画像の例では、「Dogs」と「Cat」のアイコンで示されたクラスタが、犬と猫の画像に対応しています。
*   **クエリベクトル（青い点）:** 「find me all cats」というテキストのクエリは、同様にベクトルに変換され、多次元空間内の青い点で表されます。
*   **類似度検索:** ベクトル検索では、クエリベクトルに最も近いデータベクトルを見つけます。画像では、青い点から猫のクラスタ内の赤い点への線が、この類似度を示しています。

Databricks Vector Searchにおいて、インデックスとエンドポイントは以下のように異なる役割を持っています:


### Index (インデックス)

- Vector Searchインデックスは、データのベクトル表現の集合です。
- 効率的な近似最近傍検索クエリをサポートします。
- ソースのDeltaテーブルの特定のカラムから作成されます。
- Unity Catalog内でテーブルのようなオブジェクトとして表示されます。

### Endpoint (エンドポイント)

- Vector Searchエンドポイントは、インデックスが存在するエンティティです。
- 検索リクエストを処理するためのエントリーポイントとして機能します。
- 複数のインデックスを含むことができます。
- インデックスのサイズや並列リクエストの数に応じて自動的にスケールします。
- REST APIやSDKを使用してクエリや更新に利用できます。


Vector Searchを使用する際は、まずエンドポイントを作成し、その後そのエンドポイント内に1つ以上のインデックスを作成します。エンドポイントはインデックスを管理し、クエリ処理のためのアクセスポイントとして機能します。

インデックスに格納されているデータのベクトル表現とエンドポイントは以下のように確認することができます。

In [0]:
import mlflow.deployments

# Databricks のデプロイメントクライアントを取得
deploy_client = mlflow.deployments.get_deploy_client("databricks")

# OpenAI の text-embedding-ada-002 を使用（エンベディング生成）を実行
# エンドポイント "openhack-text-embedding-ada-002" に対して、"What is Apache Spark?" という入力テキストを渡す
response = deploy_client.predict(
    endpoint="openhack-text-embedding-ada-002",
    inputs={"input": ["Apache Spark とはなんでしょう?"]}
)

# 結果を取得
embeddings = [e["embedding"] for e in response.data]
print(embeddings)

In [0]:
from databricks.vector_search.client import VectorSearchClient
vsc = VectorSearchClient()
VECTOR_SEARCH_ENDPOINT_NAME = "vs_endpoint"

# Vector Searchエンドポイントの作成には10分～15分ほどかかります。 本日は時間がないので、作成済みのエンドポイントを利用します。
# vsc.create_endpoint_and_wait("openhack-text-embedding-ada-002", VECOTR_SEARCH_ENDPOINT_NAME)

# 作成済みのエンドポイント
vsc.list_endpoints()

### ToDo: `product_documentation_vs`インデックスを作成する
Databricks Vector SearchでUnity CatalogのUIを使用して、`product_documentation`テーブルをソースとして、`product_documentation_vs`という名前のVector Indexを作成する手順は以下の通りです。

<img src="https://github.com/databricks-demos/dbdemos-resources/blob/main/images/index_creation.gif?raw=true" width="600px" style="float: right">


**手順:**

1.  **Unity Catalogへのアクセス:**

    *   Databricksワークスペースにログインします。
    *   サイドバーの「カタログ」をクリックし、Unity Catalogを開きます。

2.  **テーブルの選択:**

    *   必要なカタログとスキーマを選択して、`product_documentation`テーブルを見つけます。
    *   `product_documentation`テーブルをクリックして、詳細を表示します。

3.  **Vector Indexの作成:**

    *   画面右上の「作成」タブで、「ベクトル検索インデックス」タブを選択します。(もしこのタブが表示されていない場合、DatabricksアカウントでVector Searchが有効になっていることを確認してください。)
    *   「Create Vector Index」ボタンをクリックします。

4.  **Vector Indexの設定:**

    *   **名前:**  `product_documentation_vs`と入力します。
    *   **プライマリーキー (オプション):** 主キーとして使用する列を選択します。`id`と入力します。
    *   **エンドポイント:** リアルタイム同期またはバッチ同期を選択します。`vs_endpoint`を選択します。
    *   **同期する列:** インデックスに同期する列を選択します。`filename`と`content`を選択します。
    *   **ソース埋め込み:** 今回は文字列をインプットとして埋め込み関数を使用して埋め込み列を作成するため`コンピュート埋め込み`を選択します。`既存の埋め込み列を使用`を選択すると、ユーザー側で埋め込みを計算した列を同期します。
    *   **埋め込みモデル:** 使用する埋め込み関数を選択します。Databricksが提供する関数またはカスタムUDFを使用できます。Azure OpenAI Serviceを登録したサービングエンドポイントの`openhack-text-embedding-ada-002`を選択します。
    *   **同期モード:** 今回はバッチ同期を選択します。要件に合わせて選択してください。

5.  **Vector Indexのデプロイ:**

    *   設定を確認し、「作成」ボタンをクリックします。
    *   Vector Indexの作成プロセスが開始されます。ステータスはUIで監視できます。

In [0]:
# インデックスの作成を確認
VECTOR_SEARCH_INDEX_NAME = "product_documentation_vs"
vs_index_fullname = f"{catalog_name}.{schema_name}.{VECTOR_SEARCH_INDEX_NAME}"

index = vsc.get_index(VECTOR_SEARCH_ENDPOINT_NAME, vs_index_fullname)
index.describe()

In [0]:
# 類似コンテンツの検索
question = "バッテリー交換"

results = index.similarity_search(
  query_text=question,
  columns=["filename", "content"],
  num_results=3)
docs = results.get('result', {}).get('data_array', [])
docs

## Q3. Vector Search のツール化とアクセス制御(標準時間：10分)

### 実践例
Databricks Unity Catalog ではよく使うクエリを関数としてスキーマに登録することができます。
スキーマに登録された関数は AI Agent の部品(`UCFunctionToolkit`) として簡単に呼び出すことができます。

以下の例では、 Ringo Computer のサポート部門に寄せられたケースのうち、緊急に処理すべき新規ケースを抽出する関数を Unity Catalog へ登録しています。

登録が完了したら実際にカタログで確認してみましょう！


In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS common;

In [0]:
create_func = f"""
CREATE OR REPLACE FUNCTION {catalog_name}.common.get_high_priority_new_cases()
RETURNS TABLE (
  CaseNumber STRING,
  SuppliedName STRING,
  SuppliedEmail STRING,
  SuppliedPhone STRING,
  SuppliedCompany STRING,
  Type STRING,
  Status STRING,
  Subject STRING,
  Description STRING
)
COMMENT 'この関数は、優先順位の高い新規ケースをケース・テーブルから検索する。'
RETURN
  SELECT CaseNumber, SuppliedName, SuppliedEmail, SuppliedPhone, SuppliedCompany, Type, Status, Subject, Description 
  FROM trainer_catalog.`03_data_analysis_by_gen_ai_for_shotkotani`.case
  WHERE Status = "新規" AND Priority = "高" AND Type = "問い合わせ"
  ORDER BY CaseNumber DESC
  LIMIT 1;
"""

spark.sql(create_func)

In [0]:
%sql
-- 登録した関数を呼び出します
USE SCHEMA common;
SELECT * FROM get_high_priority_new_cases();

In [0]:
from unitycatalog.ai.core.databricks import DatabricksFunctionClient
import pandas as pd
import io

# ケースを取得する関数のフルパス
case_func_fullname = f"{catalog_name}.{schema_name}.get_high_priority_new_cases"

# DatabricksFunctionClientから関数呼び出し
client = DatabricksFunctionClient()
result = client.execute_function(case_func_fullname)

# 検索結果のPandas DataFrameによるパース
df = pd.read_csv(io.StringIO(result.value))
display(df)

### ToDo: Vector Search を行う関数`manual_retriever`をカタログへ登録する
Hint: [ドキュメント](https://docs.databricks.com/aws/ja/generative-ai/agent-framework/unstructured-retrieval-tools#unity-catalog%E6%A9%9F%E8%83%BD%E3%82%92%E6%8C%81%E3%81%A4%E3%83%99%E3%82%AF%E3%83%88%E3%83%AB%E6%A4%9C%E7%B4%A2%E3%83%AC%E3%83%88%E3%83%AA%E3%83%BC%E3%83%90%E3%83%BC%E3%83%84%E3%83%BC%E3%83%AB)  を参考にカタログへベクトル検索関数を登録しましょう

In [0]:
create_retriever = f"""
CREATE OR REPLACE FUNCTION {catalog_name}.common.manual_retriever (
  query STRING
  COMMENT 'The query string for searching our product documentation.'
) RETURNS TABLE
COMMENT 'Executes a search on product documentation to retrieve text documents most relevant to the input query.' RETURN
SELECT
  id as id,
  content
FROM
  vector_search(
    index => '{vs_index_fullname}',
    query => query,
    num_results => 3
  )
"""

spark.sql(create_retriever)

In [0]:
import mlflow
from databricks_langchain import VectorSearchRetrieverTool

vs_tool = VectorSearchRetrieverTool(
  index_name=vs_index_fullname,
  tool_name="manual_retriever",
  tool_description="Ringo Computerの製品マニュアルを検索するツールです"
)

vs_tool.invoke("バッテリー交換")